# 简单全连接神经网络

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='Nadam',
              loss=loss_fn,
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=10)
_, acc=model.evaluate(x_test,  y_test, verbose=2)
print("acc:%.3f%"%(acc))


# CNN识别手写数字

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
n,m = x_train.shape[1], x_train.shape[2]
model = Sequential()
model.add(Reshape((n,m,1), input_shape=(n,m)))
model.add(Conv2D(filters=32, kernel_size=5, activation='relu', padding="same"))
model.add(MaxPool2D())
model.add(Conv2D(filters=64, kernel_size=(5,5), activation='relu', padding="same"))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(1024,activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='Nadam',
              loss=loss_fn,
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=100)
model.evaluate(x_test,  y_test, verbose=2)

for layer in model.layers:
    print(('%s - %s')%(layer.input_shape, layer.output_shape))

Epoch 1/10


C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


600/600 [==============================] - 51s 84ms/step - loss: 0.1395 - accuracy: 0.9561
Epoch 2/10
600/600 [==============================] - 50s 83ms/step - loss: 0.0414 - accuracy: 0.9875
Epoch 3/10
600/600 [==============================] - 50s 83ms/step - loss: 0.0295 - accuracy: 0.9908
Epoch 4/10
600/600 [==============================] - 50s 83ms/step - loss: 0.0233 - accuracy: 0.9925
Epoch 5/10
600/600 [==============================] - 51s 85ms/step - loss: 0.0165 - accuracy: 0.9948
Epoch 6/10
600/600 [==============================] - 51s 85ms/step - loss: 0.0158 - accuracy: 0.9949
Epoch 7/10
600/600 [==============================] - 52s 87ms/step - loss: 0.0126 - accuracy: 0.9959
Epoch 8/10
600/600 [==============================] - 50s 83ms/step - loss: 0.0106 - accuracy: 0.9967
Epoch 9/10
600/600 [==============================] - 50s 84ms/step - loss: 0.0099 - accuracy: 0.9965
Epoch 10/10
600/600 [==============================] - 50s 84ms/step - loss: 0.0095 - accurac

# resnet手写数字识别

In [26]:
import torch
torch.__version__

'1.11.0+cpu'

# 数据预处理

In [27]:
from torchvision.models import resnet18
from torch import nn
from torch.utils.data import DataLoader

In [44]:
from torchvision.datasets import MNIST
from torchvision import transforms
my_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(3),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,0.1307,0.1307), (0.3081,0.3081,0.3081)),
    ])
# 训练集
train_ds = MNIST(
    root='mnist',
    train=True,
    download=True,
    transform=my_transform
)
# 测试集
test_ds = MNIST(
    root='mnist',
    train=False,
    download=True,
    transform=my_transform
)
train_dl = DataLoader(train_ds, batch_size=256, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=64)

# 训练

In [ ]:
from torch import optim
model = resnet18()
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 10)
model.train()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)
train_loss = 0
correct = 0
total = 0
for batch_idx,(inputs,targets) in enumerate(train_dl):
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs,targets)
    loss.backward()
    optimizer.step()
    
    train_loss+=loss.item()
    _,predicted = outputs.max(1)
    total += targets.size(0)
    correct += predicted.eq(targets).sum().item()
    print(batch_idx,len(train_dl),'Loss:%.3f | Acc:%.3f%% (%d/%d)'
         %(train_loss/(batch_idx+1),100.*correct/total,correct,total))

0 235 Loss:2.376 | Acc:10.547% (27/256)
1 235 Loss:4.501 | Acc:11.523% (59/512)
2 235 Loss:4.848 | Acc:9.505% (73/768)
3 235 Loss:4.345 | Acc:9.668% (99/1024)
4 235 Loss:4.046 | Acc:10.625% (136/1280)
5 235 Loss:3.871 | Acc:11.133% (171/1536)
6 235 Loss:3.731 | Acc:10.770% (193/1792)
7 235 Loss:3.646 | Acc:10.986% (225/2048)
8 235 Loss:3.598 | Acc:11.458% (264/2304)
9 235 Loss:3.514 | Acc:11.836% (303/2560)
10 235 Loss:3.424 | Acc:12.678% (357/2816)
11 235 Loss:3.368 | Acc:13.086% (402/3072)
12 235 Loss:3.309 | Acc:13.401% (446/3328)
13 235 Loss:3.260 | Acc:13.393% (480/3584)
14 235 Loss:3.197 | Acc:13.620% (523/3840)
15 235 Loss:3.145 | Acc:13.599% (557/4096)
16 235 Loss:3.091 | Acc:13.856% (603/4352)
17 235 Loss:3.042 | Acc:14.497% (668/4608)
18 235 Loss:2.996 | Acc:14.988% (729/4864)
19 235 Loss:2.951 | Acc:15.566% (797/5120)
20 235 Loss:2.904 | Acc:16.276% (875/5376)
21 235 Loss:2.864 | Acc:16.744% (943/5632)
22 235 Loss:2.817 | Acc:17.527% (1032/5888)
23 235 Loss:2.775 | Acc:18.06